## Traffic Crash Analysis

### Data importing and pre-processing

In [0]:
import requests
from pyspark.sql import SparkSession

# Make the GET request
resp = requests.get('https://data.cityofchicago.org/resource/85ca-t3if.json?$query=SELECT%20crash_record_id%2C%20crash_date_est_i%2C%20crash_date%2C%20posted_speed_limit%2C%20traffic_control_device%2C%20device_condition%2C%20weather_condition%2C%20lighting_condition%2C%20first_crash_type%2C%20trafficway_type%2C%20lane_cnt%2C%20alignment%2C%20roadway_surface_cond%2C%20road_defect%2C%20report_type%2C%20crash_type%2C%20intersection_related_i%2C%20private_property_i%2C%20hit_and_run_i%2C%20damage%2C%20date_police_notified%2C%20prim_contributory_cause%2C%20sec_contributory_cause%2C%20street_no%2C%20street_direction%2C%20street_name%2C%20beat_of_occurrence%2C%20photos_taken_i%2C%20statements_taken_i%2C%20dooring_i%2C%20work_zone_i%2C%20work_zone_type%2C%20workers_present_i%2C%20num_units%2C%20most_severe_injury%2C%20injuries_total%2C%20injuries_fatal%2C%20injuries_incapacitating%2C%20injuries_non_incapacitating%2C%20injuries_reported_not_evident%2C%20injuries_no_indication%2C%20injuries_unknown%2C%20crash_hour%2C%20crash_day_of_week%2C%20crash_month%2C%20latitude%2C%20longitude%2C%20location%20ORDER%20BY%20crash_date%20DESC%2C%20crash_record_id%20ASC')

# Create a Spark session
spark = SparkSession.builder.appName("SENG550").getOrCreate()

# Create a Spark DataFrame from the response text
df2 = spark.read.json(spark.sparkContext.parallelize([resp.text]))


In [0]:
# Show the DataFrame
df2.show(5)

+------------------+------------------+--------------------+----------------+-----------------+----------+-----------+--------------------+--------------------+-------------+--------------------+--------------------+---------+--------------------+-------------+--------------+-----------------------+----------------------+---------------------------+-----------------------------+--------------+----------------+----------------------+------------+--------------------+--------------------+-------------+--------------------+---------+--------------+------------------+-----------------------+------------------+--------------------+-----------+--------------------+----------------------+------------------+----------------+--------------------+---------+----------------------+--------------------+-----------------+-----------+--------------+-----------------+
|         alignment|beat_of_occurrence|          crash_date|crash_date_est_i|crash_day_of_week|crash_hour|crash_month|     crash_record

In [0]:
selected_column = df2.select("Damage")
# Extract the values from the selected column using rdd.map
rdd_from_column = selected_column.rdd.map(lambda row: row[0])

# Display the top 10 records
print("Top 10 Records from Column:")
print(rdd_from_column.take(10))

Top 10 Records from Column:
['OVER $1,500', 'OVER $1,500', '$501 - $1,500', 'OVER $1,500', 'OVER $1,500', '$501 - $1,500', 'OVER $1,500', '$501 - $1,500', 'OVER $1,500', '$501 - $1,500']
